In [1]:
import os
import glob
import h5py
import scipy.ndimage
import numpy as np

In [2]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
SOURCE_DIR = os.path.abspath('/data/aumkar/train')
TARGET_DIR = os.path.abspath('/data/aumkar/validation')
DATA_LOAD_DIR = os.path.abspath('/data/aumkar/data_load')

In [3]:
DIR_LIST1 = ['aeroplane', 'bicycle', 'horse']
DIR_LIST2 = ['knife', 'person', 'plant']
DIR_LIST3 = ['train', 'truck', 'bus']
DIR_LIST4 = ['car', 'motorcycle', 'skateboard']

In [4]:
dir_list1 = [DIR_LIST1, DIR_LIST2]
dir_list2 = [DIR_LIST3, DIR_LIST4]

In [5]:
def data_load(dirs, lab):
    
    x, y = [], []
    path = glob.glob(os.path.join(dirs, lab))
    print(path)
    for label, img in enumerate(path):
        #print(label, img)
        for i in glob.glob(os.path.join(img, '*.*')):
            #print(i, label)
            x.append(scipy.ndimage.imread(i, mode='RGB'))
            y.append(label)
    
    return np.asarray(x), np.asarray(y)

In [ ]:
x_, y_ = [], []

#Run for source data
for i in dir_list2:
    for j in i:
    
        x, y = data_load(SOURCE_DIR, j)
        print('Finished reading source data for class', j)

        x_.extend(x)
        y_.extend(y)
    
    
xsource = np.asarray(x_)
ysource = np.asarray(y_)

In [ ]:
(xsource.shape, ysource.shape)

In [ ]:
h5f = h5py.File(os.path.join(DATA_LOAD_DIR, 'xsource2.h5'), 'w')
h5f.create_dataset('xsource_2', data = xsource)

In [ ]:
h5f_ = h5py.File(os.path.join(DATA_LOAD_DIR, 'ysource2.h5'), 'w')
h5f_.create_dataset('ysource_2', data = ysource)

In [ ]:
h5f.close()

In [ ]:
h5f_.close()

In [6]:
x_t, y_t = [], []

#Run for target data
for i in dir_list1:
    for j in i:
        
        xt, yt = data_load(TARGET_DIR, j)
        print('Finished reading target data for class', j)
        x_t.extend(xt)
        y_t.extend(yt)
    
xtarget = np.asarray(x_t)
ytarget = np.asarray(y_t)

['/data/aumkar/validation/aeroplane']


/home/aumkar/anaconda3/envs/daenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.


Finished reading target data for class aeroplane
['/data/aumkar/validation/bicycle']
Finished reading target data for class bicycle
['/data/aumkar/validation/horse']
Finished reading target data for class horse
['/data/aumkar/validation/knife']
Finished reading target data for class knife
['/data/aumkar/validation/person']
Finished reading target data for class person
['/data/aumkar/validation/plant']
Finished reading target data for class plant


In [ ]:
(xtarget.shape, ytarget.shape)

In [ ]:
h5f = h5py.File(os.path.join(DATA_LOAD_DIR, 'xtarget1.h5'), 'w')
h5f.create_dataset('xtarget_1', data = xtarget)

In [ ]:
h5f_ = h5py.File(os.path.join(DATA_LOAD_DIR, 'ytarget1.h5'), 'w')
h5f_.create_dataset('ytarget_1', data = ytarget)

In [ ]:
h5f.close()

In [ ]:
h5f_.close()